In [2]:

import random
import itertools
import numpy as np
from scipy.optimize import linear_sum_assignment
import config




In [13]:


# GPT-3 XL
batch_size = 1e6 / 2048
layer_size = 24

# physical topology
num_devices = 64 
peer_delay = None
peer_bandwidth = None
regions = None

# assigned task
batch_size_per_task = 1.25e5 / 2048
layer_size_per_task = 3
send_gradient_size = 1.3 * np.dtype(np.float32).itemsize * layer_size_per_task / layer_size  # gigabytes
send_activation_size = 2024 * 2048 * np.dtype(np.float16).itemsize * batch_size_per_task / (1024 * 1024 * 1024)  # gigabytes

assert(batch_size % batch_size_per_task == 0)
assert(layer_size % layer_size_per_task == 0)
assert(num_devices == batch_size * layer_size / (batch_size_per_task * layer_size_per_task))

way = int(layer_size / layer_size_per_task) # 8
partition_size = int(batch_size / batch_size_per_task) # 8

### GA config
num_devices = 64
population_size=100
nodes=list(range(num_devices))
trails=4900
mode="default"




In [19]:
# candidate_partitions, all_cost_records, min_cost_records = GCMA(nodes=list(range(num_devices)), population_size=100, trails=4900, mode="default")
candidate_partitions = []
candidate_scores = []
candidate_min_scores = []

for i in range(population_size):
    cur_nodes = nodes.copy()    # range的node结点 复制种群数量份
    random.seed = i             # SEED
    random.shuffle(cur_nodes)   # 打乱顺序
    candidate_partitions.append(cur_nodes)  # 所有种群放在candidate_partitions中 是个list，【100，64】
    
for i, candidate_partition in enumerate(candidate_partitions):
    # 将[1...64] 变为 [1..7, 7...15,  ..... , 56...63]
    candidate_partition = [candidate_partition[i: i + partition_size] for i in range(0, num_devices, partition_size)]   
    
    # 计算dp和pp的cost
    # data_parallel_cost = compute_data_parallel_cost(candidate_partition=candidate_partition)  
    # pipeline_parallel_cost, pipeline_parallel_path, pipeline_parallel_match = compute_pipeline_parallel_cost(candidate_partition)
    
    ## 综合cost -> scores = dp + pp !!! 可以考虑一下这个改进地方
    candidate_scores.append(data_parallel_cost +2 * pipeline_parallel_cost)
    candidate_min_scores.append(np.min(candidate_scores))
    
    
    
    